# Datasets imported
DPPS

PHYSICAL

Z


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Read the CSV files
train_X = pd.read_csv('train_X.csv')
dpps = pd.read_csv('DPPS.csv', skiprows=2)
dpps= dpps.drop('AA_3', axis =1)
dpps_used= dpps[["AA_1","D7", "D8"]]
phys = pd.read_csv("Physical.csv", skiprows=2)
phys= phys.drop('AA_3', axis =1)
zs = pd.read_csv("Z-scale.csv", skiprows=2)
zs= zs.drop('AA_3', axis =1)
zs= zs.drop('Z(3)', axis =1)
vhse = pd.read_csv("VHSE-scale.csv", skiprows=2)
vhse= vhse.drop('AA_3', axis =1)
vhse_used= vhse[["AA_1","VHSE1", "VHSE2"]]

# Clean up the DPPS dataframe
dpps_used.columns = [ 'AA_1'] + [f'D{i}' for i in range(7, 9)]
dpps_dict = dpps_used.set_index('AA_1').to_dict('index')
phys_used= phys[["AA_1","Vol", "Hydro"]]
phys_dict = phys_used.set_index('AA_1').to_dict('index')
zs_dict = zs.set_index('AA_1').to_dict('index')
vhse_dict = vhse_used.set_index('AA_1').to_dict('index')

def seq_to_dpps(seq):
    result = {}
    for i, aa in enumerate(seq):
        if aa in dpps_dict:
            for j in range(7, 9):
                result[f'pos_{i+1}_D{j}'] = dpps_dict[aa][f'D{j}']
        if aa in phys_dict:
            result[f'pos_{i+1}_Phys(Hydro)'] = phys_dict[aa]["Hydro"]
        if aa in zs_dict:
            result[f'pos_{i+1}_Z(1)'] = zs_dict[aa]["Z(1)"]
            result[f'pos_{i+1}_Z(2)'] = zs_dict[aa]["Z(2)"]
        if aa in vhse_dict:
            result[f'pos_{i+1}_VHSE1'] = vhse_dict[aa]["VHSE1"]
            result[f'pos_{i+1}_VHSE2'] = vhse_dict[aa]["VHSE2"]
    return result

# Apply the function to each sequence
dpps_features = train_X['ConstructedAASeq_cln'].apply(seq_to_dpps)

# Convert the result to a DataFrame

# Use 'Id' as the target variable


In [5]:
y = pd.read_csv("train_y.csv")
Y = y.Brightness_Class

In [6]:
X = pd.DataFrame(dpps_features.tolist())

# Split the data without stratification
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))

Training set size: 24823
Test set size: 6206


In [8]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate the model
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_test)

print(f"Train accuracy: {train_score:.4f}")
print(f"Test accuracy: {test_score:.4f}")

Train accuracy: 0.9177
Test accuracy: 0.8919


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [68]:
test_x = pd.read_csv("test_X.csv")

In [111]:
dpps_features_test = test_x['ConstructedAASeq_cln'].apply(seq_to_dpps)
id = test_x.Id
X_test = pd.DataFrame(dpps_features_test.tolist())
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)
a = {"Id": id, "BrightnessClass": y_pred}
df = pd.DataFrame(a)
df.to_csv("Submit.csv", index = False)

In [9]:
from sklearn.metrics import f1_score, precision_score
f1_score(y_test, model.predict(X_test_scaled))

0.8652880947600884

In [10]:
precision_score(y_test, model.predict(X_test_scaled))

0.8401559454191033

In [14]:
X_train.to_csv("Output.csv")

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
log_reg = LogisticRegression(random_state=42, max_iter=1000)

# Define the hyperparameter grid
param_grid = {
    'C': np.logspace(-4, 4, 20),
    'penalty': ['l2', 'elasticnet'],
    'solver': [ 'lbfgs', 'liblinear', 'sag'],
    'class_weight': [None, 'balanced']
}

# Create the F1 scorer
f1_scorer = make_scorer(f1_score, average='weighted')

# Set up GridSearchCV
grid_search = GridSearchCV(b
    estimator=log_reg,
    param_grid=param_grid,
    scoring=f1_scorer,
    cv=5,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test_scaled, y_test)
print("Test set F1-score:", test_score)

# If you want to see the detailed results for all parameter combinations
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Mean F1-score: {mean_score:.3f} for {params}")

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
